# Classification using Keras

The target values must be in a column of the dataframe, and must be a numerical datatype, so we must convert them before training.

IMPORTANT: Notice that we are importing `ImageDataGenerator` from `keras_preprocessing` instead of `keras.preprocessing`, because the people at Keras are a bunch of old reactionary dinosaurs.

See the discussion [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1), and [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1).

In [3]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

## Load data and preparation

The csv files `db.csv` contains all the metadata we have extracted from wikiart: every entry corresponds to an artwork, and the `_id` column contains the filename of the image in the `data/images/` directory associated to that artwork.

In [4]:
nrows = 5000
df = pd.read_csv("../data/db.csv",nrows=nrows,na_values="?")

Train/test split the database:

In [5]:
df_train, df_test = train_test_split(df, 
                                     test_size=0.1,
                                     shuffle=True)

In [6]:
df_train.head()

,_id,artistname,genre,image,image_size_data,style,title,year
502,57727b41edc2cb3880e1007f,giovanni battista piranesi,cityscape,https://uploads2.wikiart.org/images/giovanni-b...,"[{'sizekb': 8, 'width': 210, 'height': 126, 'u...",neoclassicism,view of the facade of the basilica of st. cros...,NaN
3880,577275aaedc2cb3880ced600,maurice prendergast,nude painting (nu),https://uploads3.wikiart.org/images/maurice-pr...,"[{'sizekb': 17, 'width': 210, 'height': 221, '...",naïve art (primitivism),figures on the beach,1916-1918
2443,57728783edc2cb38800762be,gio pomodoro,sculpture,https://uploads2.wikiart.org/images/gio-pomodo...,"[{'sizekb': 3, 'width': 210, 'height': 118, 'u...",abstract art,folla,1962
4176,577278b0edc2cb3880d8b456,pierre-paul prud&#39;hon,portrait,https://uploads2.wikiart.org/images/pierre-pau...,"[{'sizekb': 8, 'width': 210, 'height': 253, 'u...",romanticism,nicolas perchet,1795
2346,57728596edc2cb388001973e,sigmar polke,abstract,https://uploads5.wikiart.org/images/sigmar-pol...,"[{'sizekb': 6, 'width': 210, 'height': 93, 'ur...",new european painting,untitled (triptych),2002


Let's initialize a dataimage generator: it is a nice interface towards many (pre)processing method in Keras, including some utilities for data augmentation.

We will use the amazing `flow_from_dataframe` function to serve the data we need.

If the files do not have an extension, run this in a shell:

    $ for f in *; do mv "$f" "$f.jpg"; done

### Classes

Decide here what feature we want to predict, and save in the `classes` set all the possible values: they are the values that appear at least once in the database.

In [18]:
feature = "style"
classes=set(df[feature])
nclass = len(classes)
print(len(classes))

80


In [19]:
classes_list= list(classes)
for i in range(10):
    print(classes_list[i])
    
#I can write a loop to access classes which are reptitive

international gothic
nan
cubism,surrealism
pop art,naïve art (primitivism)
neo-pop art
op art
cubism,expressionism
impressionism,realism
street art
art informel,minimalism


The number of classes will be needed later, the NN must know what's the output dimension.

### Image Size

The images will be scaled down to this size

In [20]:
img_size = (32,32)

## Create data generators

In [21]:
datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)
#As of now, we are not doing cross-validation. Maybe we should do that.
#Also, we should use data-augmentation here

In [22]:
train_generator = datagen.flow_from_dataframe(
                                df_train,
                                directory="../data/images/",
                                x_col="_id",
                                has_ext=False,
                                target_size=img_size,
                                y_col=feature,
                                batch_size=32,
                                classes = classes_list)

Found 4500 images belonging to 80 classes.


# Model Architecture: CNN with no tranfer learning

We'll start with the most basic architecture imaginable. 

It would be nice to use transfer learning.

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(*img_size,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nclass, activation='softmax'))


Pick an optimizer and compile the model

In [24]:
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),
              loss="categorical_crossentropy",metrics=["accuracy"])



In [25]:
#training
model.fit_generator(generator=train_generator,
                    epochs=2)

Epoch 1/2
141/141 [==============================] - 12s 85ms/step - loss: 3.2359 - acc: 0.2312
Epoch 2/2
141/141 [==============================] - 12s 88ms/step - loss: 2.7976 - acc: 0.3181


In [21]:
test_generator = datagen.flow_from_dataframe(df_test,
                                directory="../data/images/",
                                x_col="_id",has_ext=False, target_size=img_size,
                                y_col=feature,batch_size=32, classes = classes)

Found 500 images belonging to 80 classes.


# Evaluation

This returns the loss and accuracy

In [18]:
model.evaluate_generator(generator=test_generator)

[3.1134791278839113, 0.2899999997615814]

### Transfer learning

In [ ]:


model = ResNet50(weights='imagenet')